# MNIST with Convolutional Networks

In [1]:
%matplotlib inline
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math
import time
import os

In [2]:
# Process images of this size. Note that this differs from the original CIFAR
# image size of 32 x 32. If one alters this number, then the entire model
# architecture will change and any model would need to be retrained.
IMAGE_SIZE = 24

# Global constants describing the CIFAR-10 data set.
NUM_CLASSES = 10
NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 50000
NUM_EXAMPLES_PER_EPOCH_FOR_EVAL = 10000

data_dir = os.path.join('data/cifar10_data', 'cifar-10-batches-bin')

In [3]:
# Parameters
learning_rate = 0.001
training_epochs = 100
batch_size = 128
display_step = 1

In [4]:
def read_cifar10(filename_queue):
    """Reads and parses examples from CIFAR10 data files.
    
    Recommendation: if you want N-way read parallelism, call this function
    N times.  This will give you N independent Readers reading different
    files & positions within those files, which will give better mixing of
    examples.
    
    Args:
      filename_queue: A queue of strings with the filenames to read from.
      
    Returns:
      An object representing a single example, with the following fields:
        height: number of rows in the result (32)
        width: number of columns in the result (32)
        depth: number of color channels in the result (3)
        key: a scalar string Tensor describing the filename & record number
          for this example.
        label: an int32 Tensor with the label in the range 0..9.
        uint8image: a [height, width, depth] uint8 Tensor with the image data
    """

    class CIFAR10Record(object):
        pass
    result = CIFAR10Record()

    # Dimensions of the images in the CIFAR-10 dataset.
    # See http://www.cs.toronto.edu/~kriz/cifar.html for a description of the
    # input format.
    label_bytes = 1  # 2 for CIFAR-100
    result.height = 32
    result.width = 32
    result.depth = 3
    image_bytes = result.height * result.width * result.depth
    # Every record consists of a label followed by the image, with a
    # fixed number of bytes for each.
    record_bytes = label_bytes + image_bytes

    # Read a record, getting filenames from the filename_queue.  No
    # header or footer in the CIFAR-10 format, so we leave header_bytes
    # and footer_bytes at their default of 0.
    reader = tf.FixedLengthRecordReader(record_bytes=record_bytes)
    result.key, value = reader.read(filename_queue)

    # Convert from a string to a vector of uint8 that is record_bytes long.
    record_bytes = tf.decode_raw(value, tf.uint8)

    # The first bytes represent the label, which we convert from uint8->int32.
    result.label = tf.cast(
        tf.strided_slice(record_bytes, [0], [label_bytes]), tf.int32)

    # The remaining bytes after the label represent the image, which we reshape
    # from [depth * height * width] to [depth, height, width].
    depth_major = tf.reshape(
        tf.strided_slice(record_bytes, [label_bytes],
                         [label_bytes + image_bytes]),
        [result.depth, result.height, result.width])
    # Convert from [depth, height, width] to [height, width, depth].
    result.uint8image = tf.transpose(depth_major, [1, 2, 0])

    return result

In [5]:
def _generate_image_and_label_batch(image, label, min_queue_examples, batch_size, shuffle):
    """Construct a queued batch of images and labels.
    
    Args:
      image: 3-D Tensor of [height, width, 3] of type.float32.
      label: 1-D Tensor of type.int32
      min_queue_examples: int32, minimum number of samples to retain
      in the queue that provides of batches of examples.
      batch_size: Number of images per batch.
      
    Returns:
      images: Images. 4D tensor of [batch_size, height, width, 3] size.
      labels: Labels. 1D tensor of [batch_size] size.
    """
    # Create a queue that shuffles the examples, and then
    # read 'batch_size' images + labels from the example queue.
    num_preprocess_threads = 16
    if shuffle:
        images, label_batch = tf.train.shuffle_batch(
            [image, label],
            batch_size=batch_size,
            num_threads=num_preprocess_threads,
            capacity=min_queue_examples + 3 * batch_size,
            min_after_dequeue=min_queue_examples)
    else:
        images, label_batch = tf.train.batch(
            [image, label],
            batch_size=batch_size,
            num_threads=num_preprocess_threads,
            capacity=min_queue_examples + 3 * batch_size)

    # Display the training images in the visualizer.
    tf.summary.image('images', images)

    return images, tf.reshape(label_batch, [batch_size])

In [6]:
def distorted_inputs(data_dir, batch_size):
    """Construct distorted input for CIFAR training using the Reader ops.
    Args:
      data_dir: Path to the CIFAR-10 data directory.
      batch_size: Number of images per batch.
    Returns:
      images: Images. 4D tensor of [batch_size, IMAGE_SIZE, IMAGE_SIZE, 3] size.
      labels: Labels. 1D tensor of [batch_size] size.
    """

    filenames = [os.path.join(data_dir, 'data_batch_%d.bin' % i) 
                 for i in range(1, 6)]
    for f in filenames:
        if not tf.gfile.Exists(f):
            raise ValueError('Failed to find file: ' + f)

    # Create a queue that produces the filenames to read.
    filename_queue = tf.train.string_input_producer(filenames)

    # Read examples from files in the filename queue.
    read_input = read_cifar10(filename_queue)
    reshaped_image = tf.cast(read_input.uint8image, tf.float32)
    
    height = IMAGE_SIZE
    width = IMAGE_SIZE

    # Image processing for training the network. Note the many random
    # distortions applied to the image.

    # Randomly crop a [height, width] section of the image.
    distorted_image = tf.random_crop(reshaped_image, size=[height, width, 3])

    # Randomly flip the image horizontally.
    distorted_image = tf.image.random_flip_left_right(distorted_image)

    # Because these operations are not commutative, consider randomizing
    # randomize the order their operation.
    distorted_image = tf.image.random_brightness(distorted_image, max_delta=63)
    distorted_image = tf.image.random_contrast(distorted_image, lower=0.2, upper=1.8)

    # Subtract off the mean and divide by the variance of the pixels.
    float_image = tf.image.per_image_standardization(distorted_image)
    
    # Set the shapes of tensors.
    float_image.set_shape([height, width, 3])
    read_input.label.set_shape([1])

    # Ensure that the random shuffling has good mixing properties.
    min_fraction_of_examples_in_queue = 0.4
    min_queue_examples = int(NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN *
                           min_fraction_of_examples_in_queue)
    print ('Filling queue with %d CIFAR images before starting to train. '
           'This will take a few minutes.' % min_queue_examples)

    # Generate a batch of images and labels by building up a queue of examples.

    return _generate_image_and_label_batch(float_image, read_input.label, min_queue_examples, batch_size, shuffle=True)

In [7]:
def inputs(eval_data, data_dir, batch_size):
    """Construct input for CIFAR evaluation using the Reader ops.
    Args:
      eval_data: bool, indicating if one should use the train or eval data set.
      data_dir: Path to the CIFAR-10 data directory.
      batch_size: Number of images per batch.
    Returns:
      images: Images. 4D tensor of [batch_size, IMAGE_SIZE, IMAGE_SIZE, 3] size.
      labels: Labels. 1D tensor of [batch_size] size.
    """
    if not eval_data:
        filenames = [os.path.join(data_dir, 'data_batch_%d.bin' % i) for i in xrange(1, 6)]
        num_examples_per_epoch = NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN
    else:
        filenames = [os.path.join(data_dir, 'test_batch.bin')]
        num_examples_per_epoch = NUM_EXAMPLES_PER_EPOCH_FOR_EVAL

    for f in filenames:
        if not tf.gfile.Exists(f):
            raise ValueError('Failed to find file: ' + f)

    with tf.name_scope('input'):
        # Create a queue that produces the filenames to read.
        filename_queue = tf.train.string_input_producer(filenames)

        # Read examples from files in the filename queue.
        read_input = read_cifar10(filename_queue)
        reshaped_image = tf.cast(read_input.uint8image, tf.float32)

        height = IMAGE_SIZE
        width = IMAGE_SIZE

        # Image processing for evaluation.
        # Crop the central [height, width] of the image.
        resized_image = tf.image.resize_image_with_crop_or_pad(reshaped_image, height, width)

        # Subtract off the mean and divide by the variance of the pixels.
        float_image = tf.image.per_image_standardization(resized_image)

        # Set the shapes of tensors.
        float_image.set_shape([height, width, 3])
        read_input.label.set_shape([1])

        # Ensure that the random shuffling has good mixing properties.
        min_fraction_of_examples_in_queue = 0.4
        min_queue_examples = int(num_examples_per_epoch * min_fraction_of_examples_in_queue)

    # Generate a batch of images and labels by building up a queue of examples.
    return _generate_image_and_label_batch(float_image, read_input.label, min_queue_examples, batch_size, shuffle=False)

In [8]:
# Architecture
n_hidden_1 = 256
n_hidden_2 = 256

In [9]:
def filter_summary(V, weight_shape):
    ix = weight_shape[0]
    iy = weight_shape[1]
    cx, cy = 8, 8
    V_T = tf.transpose(V, (3, 0, 1, 2))
    tf.summary.image("filters", V_T, max_outputs=64)

In [10]:
def conv2d(input, weight_shape, bias_shape, visualize=False):
    incoming = weight_shape[0] * weight_shape[1] * weight_shape[2]
    weight_init = tf.random_normal_initializer(stddev=(2.0/incoming)**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    if visualize:
        filter_summary(W, weight_shape)
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)
    logits = tf.nn.bias_add(tf.nn.conv2d(input, W, strides=[1, 1, 1, 1], padding='SAME'), b)
    return tf.nn.relu(logits)

In [11]:
def max_pool(input, k=2):
    return tf.nn.max_pool(input, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [12]:
def layer(input, weight_shape, bias_shape):
    weight_stddev = (2.0/weight_shape[0])**0.5
    w_init = tf.random_normal_initializer(stddev=weight_stddev)
    bias_init = tf.constant_initializer(value=0)
    W = tf.get_variable("W", weight_shape, initializer=w_init)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)
    logits = tf.matmul(input, W) + b
    return tf.nn.relu(logits)

In [13]:
def inference(x, keep_prob):

    with tf.variable_scope("conv_1"):
        conv_1 = conv2d(x, [5, 5, 3, 64], [64], visualize=True)
        pool_1 = max_pool(conv_1)

    with tf.variable_scope("conv_2"):
        conv_2 = conv2d(pool_1, [5, 5, 64, 64], [64])
        pool_2 = max_pool(conv_2)

    with tf.variable_scope("fc_1"):

        dim = 1
        for d in pool_2.get_shape()[1:].as_list():
            dim *= d

        pool_2_flat = tf.reshape(pool_2, [-1, dim])
        fc_1 = layer(pool_2_flat, [dim, 384], [384])
        
        # apply dropout
        fc_1_drop = tf.nn.dropout(fc_1, keep_prob)

    with tf.variable_scope("fc_2"):

        fc_2 = layer(fc_1_drop, [384, 192], [192])
        
        # apply dropout
        fc_2_drop = tf.nn.dropout(fc_2, keep_prob)

    with tf.variable_scope("output"):
        output = layer(fc_2_drop, [192, 10], [10])

    return output

In [14]:
def loss(output, y):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output, labels=tf.cast(y, tf.int64))
    loss = tf.reduce_mean(xentropy)    
    return loss

In [15]:
def training(cost, global_step):
    tf.summary.scalar("cost", cost)
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op

In [16]:
def evaluate(output, y):
    correct_prediction = tf.equal(tf.cast(tf.argmax(output, 1), dtype=tf.int32), y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("validation error", (1.0 - accuracy))
    return accuracy

In [17]:
if __name__ == '__main__':

    with tf.device("/gpu:0"):

        with tf.Graph().as_default():

            with tf.variable_scope("cifar_conv_model"):

                x = tf.placeholder("float", [None, 24, 24, 3])
                y = tf.placeholder("int32", [None])
                keep_prob = tf.placeholder(tf.float32) # dropout probability

                distorted_images, distorted_labels = distorted_inputs(data_dir, batch_size)
                val_images, val_labels = inputs(True, data_dir, batch_size)

                output = inference(x, keep_prob)

                cost = loss(output, y)

                global_step = tf.Variable(0, name='global_step', trainable=False)

                train_op = training(cost, global_step)

                eval_op = evaluate(output, y)

                summary_op = tf.summary.merge_all()

                saver = tf.train.Saver()

                sess = tf.Session()

                summary_writer = tf.summary.FileWriter("conv_cifar_logs/", graph_def=sess.graph_def)

                
                init_op = tf.global_variables_initializer()

                sess.run(init_op)
                
                tf.train.start_queue_runners(sess=sess)

                # Training cycle
                for epoch in range(training_epochs):
                    
                    avg_cost = 0.
                    total_batch = int(NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN/batch_size)
                    # Loop over all batches
                    for i in range(total_batch):
                        # Fit training using batch data
                        
                        train_x, train_y = sess.run([distorted_images, distorted_labels])
                        
                        _, new_cost = sess.run([train_op, cost], feed_dict={x: train_x, y: train_y, keep_prob: 0.5})                                            
                        # Compute average loss
                        avg_cost += new_cost/total_batch
                        print("Epoch %d, minibatch %d of %d. Cost = %0.4f." %(epoch, i, total_batch, new_cost))
                             
                    # Display logs per epoch step
                    if epoch % display_step == 0:
                        print("Epoch:", '%04d' % (epoch+1), "cost =", "{:.9f}".format(avg_cost))
                        
                        val_x, val_y = sess.run([val_images, val_labels])
                        
                        accuracy = sess.run(eval_op, feed_dict={x: val_x, y: val_y, keep_prob: 1})

                        print("Validation Error:", (1 - accuracy))

                        summary_str = sess.run(summary_op, feed_dict={x: train_x, y: train_y, keep_prob: 1})
                        summary_writer.add_summary(summary_str, sess.run(global_step))

                        saver.save(sess, "conv_cifar_logs/model-checkpoint", global_step=global_step)
                        
                print("Optimization Finished!")
                
                val_x, val_y = sess.run([val_images, val_labels])
                accuracy = sess.run(eval_op, feed_dict={x: val_x, y: val_y, keep_prob: 1})

                print("Test Accuracy:", accuracy)
                        

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.
INFO:tensorflow:Summary name validation error is illegal; using validation_error instead.
Epoch 0, minibatch 0 of 390. Cost = 5.3609.
Epoch 0, minibatch 1 of 390. Cost = 4.0839.
Epoch 0, minibatch 2 of 390. Cost = 2.9928.
Epoch 0, minibatch 3 of 390. Cost = 2.8313.
Epoch 0, minibatch 4 of 390. Cost = 2.4525.
Epoch 0, minibatch 5 of 390. Cost = 2.4044.
Epoch 0, minibatch 6 of 390. Cost = 2.3311.
Epoch 0, minibatch 7 of 390. Cost = 2.2996.
Epoch 0, minibatch 8 of 390. Cost = 2.2954.
Epoch 0, minibatch 9 of 390. Cost = 2.3085.
Epoch 0, minibatch 10 of 390. Cost = 2.2781.
Epoch 0, minibatch 11 of 390. Cost = 2.2972.
Epoch 0, minibatch 12 of 390. Cost = 2.3043.
Epoch 0, minibatch 13 of 390. Cost = 2.3143.
Epoch 0, minibatch 14 of 390. Cost = 2.2907.
Epoch 0, minibatch 15 of 390. Cost = 2.2904.
Epoch 0, minibatch 16 of 390. Cost = 2.2896.
Epoch 0, minibatch 17 of 390. Cost = 2.2909.
Epoch 0, minibat

Epoch 0, minibatch 175 of 390. Cost = 1.9750.
Epoch 0, minibatch 176 of 390. Cost = 2.0500.
Epoch 0, minibatch 177 of 390. Cost = 2.0532.
Epoch 0, minibatch 178 of 390. Cost = 1.9545.
Epoch 0, minibatch 179 of 390. Cost = 1.9004.
Epoch 0, minibatch 180 of 390. Cost = 1.9455.
Epoch 0, minibatch 181 of 390. Cost = 1.8752.
Epoch 0, minibatch 182 of 390. Cost = 2.0322.
Epoch 0, minibatch 183 of 390. Cost = 2.0143.
Epoch 0, minibatch 184 of 390. Cost = 1.9738.
Epoch 0, minibatch 185 of 390. Cost = 1.9933.
Epoch 0, minibatch 186 of 390. Cost = 2.0725.
Epoch 0, minibatch 187 of 390. Cost = 1.9104.
Epoch 0, minibatch 188 of 390. Cost = 1.8926.
Epoch 0, minibatch 189 of 390. Cost = 2.0403.
Epoch 0, minibatch 190 of 390. Cost = 1.9849.
Epoch 0, minibatch 191 of 390. Cost = 1.9751.
Epoch 0, minibatch 192 of 390. Cost = 1.9980.
Epoch 0, minibatch 193 of 390. Cost = 1.9630.
Epoch 0, minibatch 194 of 390. Cost = 1.9836.
Epoch 0, minibatch 195 of 390. Cost = 1.7959.
Epoch 0, minibatch 196 of 390. Cos

Epoch 0, minibatch 353 of 390. Cost = 1.7599.
Epoch 0, minibatch 354 of 390. Cost = 1.6104.
Epoch 0, minibatch 355 of 390. Cost = 1.7682.
Epoch 0, minibatch 356 of 390. Cost = 1.7479.
Epoch 0, minibatch 357 of 390. Cost = 1.7226.
Epoch 0, minibatch 358 of 390. Cost = 1.7773.
Epoch 0, minibatch 359 of 390. Cost = 1.6295.
Epoch 0, minibatch 360 of 390. Cost = 1.7408.
Epoch 0, minibatch 361 of 390. Cost = 1.8420.
Epoch 0, minibatch 362 of 390. Cost = 1.7231.
Epoch 0, minibatch 363 of 390. Cost = 1.8586.
Epoch 0, minibatch 364 of 390. Cost = 1.6519.
Epoch 0, minibatch 365 of 390. Cost = 1.5671.
Epoch 0, minibatch 366 of 390. Cost = 1.8505.
Epoch 0, minibatch 367 of 390. Cost = 1.7622.
Epoch 0, minibatch 368 of 390. Cost = 1.8040.
Epoch 0, minibatch 369 of 390. Cost = 1.8315.
Epoch 0, minibatch 370 of 390. Cost = 1.6623.
Epoch 0, minibatch 371 of 390. Cost = 1.8033.
Epoch 0, minibatch 372 of 390. Cost = 1.7118.
Epoch 0, minibatch 373 of 390. Cost = 1.9377.
Epoch 0, minibatch 374 of 390. Cos

Epoch 1, minibatch 142 of 390. Cost = 1.7471.
Epoch 1, minibatch 143 of 390. Cost = 1.6983.
Epoch 1, minibatch 144 of 390. Cost = 1.8063.
Epoch 1, minibatch 145 of 390. Cost = 1.6275.
Epoch 1, minibatch 146 of 390. Cost = 1.8159.
Epoch 1, minibatch 147 of 390. Cost = 1.6987.
Epoch 1, minibatch 148 of 390. Cost = 1.8375.
Epoch 1, minibatch 149 of 390. Cost = 1.6837.
Epoch 1, minibatch 150 of 390. Cost = 1.6277.
Epoch 1, minibatch 151 of 390. Cost = 1.5466.
Epoch 1, minibatch 152 of 390. Cost = 1.5695.
Epoch 1, minibatch 153 of 390. Cost = 1.6352.
Epoch 1, minibatch 154 of 390. Cost = 1.6705.
Epoch 1, minibatch 155 of 390. Cost = 1.6292.
Epoch 1, minibatch 156 of 390. Cost = 1.5668.
Epoch 1, minibatch 157 of 390. Cost = 1.6207.
Epoch 1, minibatch 158 of 390. Cost = 1.7865.
Epoch 1, minibatch 159 of 390. Cost = 1.6344.
Epoch 1, minibatch 160 of 390. Cost = 1.6076.
Epoch 1, minibatch 161 of 390. Cost = 1.5913.
Epoch 1, minibatch 162 of 390. Cost = 1.7214.
Epoch 1, minibatch 163 of 390. Cos

Epoch 1, minibatch 320 of 390. Cost = 1.6177.
Epoch 1, minibatch 321 of 390. Cost = 1.6031.
Epoch 1, minibatch 322 of 390. Cost = 1.6342.
Epoch 1, minibatch 323 of 390. Cost = 1.6413.
Epoch 1, minibatch 324 of 390. Cost = 1.7281.
Epoch 1, minibatch 325 of 390. Cost = 1.6562.
Epoch 1, minibatch 326 of 390. Cost = 1.6726.
Epoch 1, minibatch 327 of 390. Cost = 1.4796.
Epoch 1, minibatch 328 of 390. Cost = 1.6859.
Epoch 1, minibatch 329 of 390. Cost = 1.6420.
Epoch 1, minibatch 330 of 390. Cost = 1.7562.
Epoch 1, minibatch 331 of 390. Cost = 1.6831.
Epoch 1, minibatch 332 of 390. Cost = 1.6346.
Epoch 1, minibatch 333 of 390. Cost = 1.5307.
Epoch 1, minibatch 334 of 390. Cost = 1.8072.
Epoch 1, minibatch 335 of 390. Cost = 1.5602.
Epoch 1, minibatch 336 of 390. Cost = 1.5415.
Epoch 1, minibatch 337 of 390. Cost = 1.4867.
Epoch 1, minibatch 338 of 390. Cost = 1.4946.
Epoch 1, minibatch 339 of 390. Cost = 1.6199.
Epoch 1, minibatch 340 of 390. Cost = 1.4526.
Epoch 1, minibatch 341 of 390. Cos

...

Epoch 98, minibatch 12 of 390. Cost = 0.6398.
Epoch 98, minibatch 13 of 390. Cost = 0.7652.
Epoch 98, minibatch 14 of 390. Cost = 0.8282.
Epoch 98, minibatch 15 of 390. Cost = 0.6298.
Epoch 98, minibatch 16 of 390. Cost = 0.6425.
Epoch 98, minibatch 17 of 390. Cost = 0.6993.
Epoch 98, minibatch 18 of 390. Cost = 0.7833.
Epoch 98, minibatch 19 of 390. Cost = 0.5755.
Epoch 98, minibatch 20 of 390. Cost = 0.7370.
Epoch 98, minibatch 21 of 390. Cost = 0.7014.
Epoch 98, minibatch 22 of 390. Cost = 0.7375.
Epoch 98, minibatch 23 of 390. Cost = 0.7972.
Epoch 98, minibatch 24 of 390. Cost = 0.4811.
Epoch 98, minibatch 25 of 390. Cost = 0.6947.
Epoch 98, minibatch 26 of 390. Cost = 0.7613.
Epoch 98, minibatch 27 of 390. Cost = 0.7363.
Epoch 98, minibatch 28 of 390. Cost = 0.6468.
Epoch 98, minibatch 29 of 390. Cost = 0.8501.
Epoch 98, minibatch 30 of 390. Cost = 0.6333.
Epoch 98, minibatch 31 of 390. Cost = 0.5352.
Epoch 98, minibatch 32 of 390. Cost = 0.7603.
Epoch 98, minibatch 33 of 390. Cos

Epoch 98, minibatch 190 of 390. Cost = 0.6322.
Epoch 98, minibatch 191 of 390. Cost = 0.8229.
Epoch 98, minibatch 192 of 390. Cost = 0.7122.
Epoch 98, minibatch 193 of 390. Cost = 0.7493.
Epoch 98, minibatch 194 of 390. Cost = 0.7507.
Epoch 98, minibatch 195 of 390. Cost = 0.7622.
Epoch 98, minibatch 196 of 390. Cost = 0.9091.
Epoch 98, minibatch 197 of 390. Cost = 0.6669.
Epoch 98, minibatch 198 of 390. Cost = 0.8441.
Epoch 98, minibatch 199 of 390. Cost = 0.8208.
Epoch 98, minibatch 200 of 390. Cost = 0.9690.
Epoch 98, minibatch 201 of 390. Cost = 0.9486.
Epoch 98, minibatch 202 of 390. Cost = 0.6579.
Epoch 98, minibatch 203 of 390. Cost = 0.6808.
Epoch 98, minibatch 204 of 390. Cost = 0.8168.
Epoch 98, minibatch 205 of 390. Cost = 0.7870.
Epoch 98, minibatch 206 of 390. Cost = 0.7857.
Epoch 98, minibatch 207 of 390. Cost = 0.6566.
Epoch 98, minibatch 208 of 390. Cost = 0.7409.
Epoch 98, minibatch 209 of 390. Cost = 0.7560.
Epoch 98, minibatch 210 of 390. Cost = 0.6856.
Epoch 98, min

Epoch 98, minibatch 365 of 390. Cost = 0.8071.
Epoch 98, minibatch 366 of 390. Cost = 0.8649.
Epoch 98, minibatch 367 of 390. Cost = 0.6550.
Epoch 98, minibatch 368 of 390. Cost = 0.6014.
Epoch 98, minibatch 369 of 390. Cost = 0.9136.
Epoch 98, minibatch 370 of 390. Cost = 0.7804.
Epoch 98, minibatch 371 of 390. Cost = 0.7721.
Epoch 98, minibatch 372 of 390. Cost = 0.6988.
Epoch 98, minibatch 373 of 390. Cost = 0.8447.
Epoch 98, minibatch 374 of 390. Cost = 0.6856.
Epoch 98, minibatch 375 of 390. Cost = 0.7208.
Epoch 98, minibatch 376 of 390. Cost = 0.6727.
Epoch 98, minibatch 377 of 390. Cost = 0.8111.
Epoch 98, minibatch 378 of 390. Cost = 0.6440.
Epoch 98, minibatch 379 of 390. Cost = 0.7303.
Epoch 98, minibatch 380 of 390. Cost = 0.6463.
Epoch 98, minibatch 381 of 390. Cost = 1.0435.
Epoch 98, minibatch 382 of 390. Cost = 0.8264.
Epoch 98, minibatch 383 of 390. Cost = 0.7167.
Epoch 98, minibatch 384 of 390. Cost = 0.9791.
Epoch 98, minibatch 385 of 390. Cost = 0.7212.
Epoch 98, min

Epoch 99, minibatch 151 of 390. Cost = 0.6751.
Epoch 99, minibatch 152 of 390. Cost = 0.7457.
Epoch 99, minibatch 153 of 390. Cost = 0.8039.
Epoch 99, minibatch 154 of 390. Cost = 0.6496.
Epoch 99, minibatch 155 of 390. Cost = 0.7853.
Epoch 99, minibatch 156 of 390. Cost = 0.7756.
Epoch 99, minibatch 157 of 390. Cost = 0.9135.
Epoch 99, minibatch 158 of 390. Cost = 0.8163.
Epoch 99, minibatch 159 of 390. Cost = 0.8016.
Epoch 99, minibatch 160 of 390. Cost = 0.7167.
Epoch 99, minibatch 161 of 390. Cost = 0.6742.
Epoch 99, minibatch 162 of 390. Cost = 0.6759.
Epoch 99, minibatch 163 of 390. Cost = 0.7338.
Epoch 99, minibatch 164 of 390. Cost = 0.6938.
Epoch 99, minibatch 165 of 390. Cost = 0.7842.
Epoch 99, minibatch 166 of 390. Cost = 0.5475.
Epoch 99, minibatch 167 of 390. Cost = 0.9093.
Epoch 99, minibatch 168 of 390. Cost = 0.7629.
Epoch 99, minibatch 169 of 390. Cost = 0.6448.
Epoch 99, minibatch 170 of 390. Cost = 0.7500.
Epoch 99, minibatch 171 of 390. Cost = 0.7716.
Epoch 99, min

Epoch 99, minibatch 326 of 390. Cost = 0.7938.
Epoch 99, minibatch 327 of 390. Cost = 0.6393.
Epoch 99, minibatch 328 of 390. Cost = 0.9153.
Epoch 99, minibatch 329 of 390. Cost = 0.7482.
Epoch 99, minibatch 330 of 390. Cost = 0.7711.
Epoch 99, minibatch 331 of 390. Cost = 0.7080.
Epoch 99, minibatch 332 of 390. Cost = 0.8862.
Epoch 99, minibatch 333 of 390. Cost = 0.7011.
Epoch 99, minibatch 334 of 390. Cost = 0.6273.
Epoch 99, minibatch 335 of 390. Cost = 0.8652.
Epoch 99, minibatch 336 of 390. Cost = 0.5171.
Epoch 99, minibatch 337 of 390. Cost = 0.7825.
Epoch 99, minibatch 338 of 390. Cost = 0.9065.
Epoch 99, minibatch 339 of 390. Cost = 0.6597.
Epoch 99, minibatch 340 of 390. Cost = 0.6688.
Epoch 99, minibatch 341 of 390. Cost = 0.8243.
Epoch 99, minibatch 342 of 390. Cost = 0.6851.
Epoch 99, minibatch 343 of 390. Cost = 1.0034.
Epoch 99, minibatch 344 of 390. Cost = 0.7996.
Epoch 99, minibatch 345 of 390. Cost = 0.8165.
Epoch 99, minibatch 346 of 390. Cost = 0.6846.
Epoch 99, min